In [180]:
import geopandas as gpd
import pandas as pd
import numpy as np
import re
from collections import Counter
from tqdm import tqdm

# Street Data

## Preprocessing (from other Notebook)

In [181]:
voies = pd.read_csv("data/opendata_voie_paris.csv", sep=";")
#keep only columns that might be useful further on
voies = voies[["N_SQ_VO", "L_VOIE", "L_COURTMIN", "L_LONGMIN"]]

#preprocessing
def preprocess(df, column, map_dict):
    data = df.copy()
    new_colname = f"{column}_prep"
    data[new_colname] = data[column].str.lower()
    data[new_colname] = data[new_colname].replace(map_dict, regex=True)
    return data

map_dict = {"é": "e", "è": "e", "ê":"e", "à":"a", "â":"a", "ô":"o", "î":"i", "û":"u", "ç":"c", "\-":" ", "\_":" "}

#apply preprocessing to voies
voies = preprocess(voies, "L_VOIE", map_dict)
voies = preprocess(voies, "L_COURTMIN", map_dict)
voies = preprocess(voies, "L_LONGMIN", map_dict)

#add column to remember data source
voies["source"] = "2022"

In [182]:
old = gpd.read_file("data/1791.zip", encoding='utf-8')
old = old.drop(["geometry"], axis=1)
# drop rows that don't have street names
old = old.dropna(subset=["nom_voie"])
# replace None values by string ""
old = old.fillna("")
old["voie"] = old["type_voie"] + " " + old["particule"] + " " + old["nom_voie"]
old = old.drop(["type_voie", "particule"], axis=1)
old = preprocess(old, "voie", map_dict)
old = preprocess(old, "nom_voie", map_dict)

#add column to remember data source
old["source"] = "1791"

In [183]:
streets = old.merge(voies, how='outer', left_on="voie_prep", right_on="L_LONGMIN_prep")
print("#total:", len(streets), "#old:", len(old), "#new:", len(voies), "#both old and new:", len(streets.dropna()))
streets["streetname"] = streets["voie_prep"].fillna(streets["L_LONGMIN_prep"])
# add a column to know where the data came from
streets["source"] = streets["source_x"]
streets.loc[streets["source_y"].notna(), "source"] = streets["source_y"]
streets.loc[(streets["source_y"].notna() & streets["source_x"].notna()), "source"] = "both"
streets = streets.drop(["source_x", "source_y"], axis=1)
# add a column with short streetnames
streets["streetname_short"] = streets["L_VOIE_prep"]
streets["streetname_short"] = streets["streetname_short"].fillna(streets["nom_voie_prep"])
streets

#total: 7630 #old: 1292 #new: 6542 #both old and new: 229


,id,nom_voie,voie,voie_prep,nom_voie_prep,N_SQ_VO,L_VOIE,L_COURTMIN,L_LONGMIN,L_VOIE_prep,L_COURTMIN_prep,L_LONGMIN_prep,streetname,source,streetname_short
0,1162.0,deux Boules,rue des deux Boules,rue des deux boules,deux boules,750005865.0,DEUX BOULES,R. des Deux Boules,Rue des Deux Boules,deux boules,r. des deux boules,rue des deux boules,rue des deux boules,both,deux boules
1,1531.0,Jean Lantier,rue Jean Lantier,rue jean lantier,jean lantier,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rue jean lantier,1791,jean lantier
2,3.0,Orfèvres,rue des Orfèvres,rue des orfevres,orfevres,750006633.0,ORFEVRES,R. des Orfèvres,Rue des Orfèvres,orfevres,r. des orfevres,rue des orfevres,rue des orfevres,both,orfevres
3,4.0,mauvaises Paroles,rue des mauvaises Paroles,rue des mauvaises paroles,mauvaises paroles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rue des mauvaises paroles,1791,mauvaises paroles
4,5.0,Plat d'Etain,rue du Plat d'Etain,rue du plat d'etain,plat d'etain,750005668.0,PLAT D'ETAIN,R. du Plat d'Etain,Rue du Plat d'Etain,plat d'etain,r. du plat d'etain,rue du plat d'etain,rue du plat d'etain,both,plat d'etain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7625,NaN,NaN,NaN,NaN,NaN,750005143.0,PETITOT,R. Petitot,Rue Petitot,petitot,r. petitot,rue petitot,rue petitot,2022,petitot
7626,NaN,NaN,NaN,NaN,NaN,750006122.0,PAUL-HENRI GRAUWIN,R. Paul-Henri Grauwin,Rue Paul-Henri Grauwin,paul henri grauwin,r. paul henri grauwin,rue paul henri grauwin,rue paul henri grauwin,2022,paul henri grauwin
7627,NaN,NaN,NaN,NaN,NaN,750006676.0,BF/17,Voie Bf/17,Voie Bf/17,bf/17,voie bf/17,voie bf/17,voie bf/17,2022,bf/17
7628,NaN,NaN,NaN,NaN,NaN,750003865.0,CHARLES CROS,R. Charles Cros,Rue Charles Cros,charles cros,r. charles cros,rue charles cros,rue charles cros,2022,charles cros


# Bottin Data
## Preprocessing

In [184]:
#compute prefix dictionary with help of voies data
def get_prefix(row, court, long):
    return row[long].split(row[court])[0]

voies["prefix_court"] = voies.apply(get_prefix, args=("L_VOIE_prep", "L_COURTMIN_prep"), axis=1)
voies["prefix_long"] = voies.apply(get_prefix, args=("L_VOIE_prep", "L_LONGMIN_prep"), axis=1)
prefix_dict = dict(zip(voies["prefix_court"], voies["prefix_long"]))
# only get prefixes with . in it
pfx_select = {key.split(" ")[0]:value.split(" ")[0] for key,value in prefix_dict.items() if "." in key}

pfx_select["boul."] = "boulevard"
pfx_select["boulev."] = "boulevard"
pfx_select["boulv."] = "boulevard"
pfx_select["q."] = "quai"
pfx_select["aven."] = "avenue"
pfx_select["faub."] = "faubourg"
pfx_select["fau."] = "faubourg"
pfx_select["st."] =  "saint"
pfx_select["impas."] = "impasse"
pfx_select["l'aub."] = "l'auberge"
pfx_select["laub"] = "l'auberge"
pfx_select["st"] = "saint"
pfx_select["ste"] = "sainte"
pfx_select["sts"] = "saints"
pfx_select["nve"] = "neuve"


In [185]:
# load bottin data and preprocess
# change as many abbreviations as possible

bottins = pd.read_csv("data/strict_addressing.csv")
bottins["Rue"] = bottins["Rue"].fillna("")
bottins = preprocess(bottins, "Rue", map_dict)

def get_abbreviation(street):
    # returns streets which include a "."
    try:
        if "." in street:
            return street
    except:
        print(street)
      
# get number of abbreviations
print("#abbr. in data before dict substitution:", len(set([get_abbreviation(street) for street in list(bottins["Rue_prep"])])))
# substitute abbreviations
bottins["Rue_no_abbr"] = bottins["Rue_prep"].str.split().apply(lambda x: ' '.join([pfx_select.get(e, e) for e in x]))
# see how many abbreviations are left
print("#abbr. in data after dict substitution:", len(set([get_abbreviation(street) for street in list(bottins["Rue_no_abbr"])])))

#abbr. in data before dict substitution: 76567
#abbr. in data after dict substitution: 37883


## Align data

In [186]:
def align_on_column(df_not_aligned, df_streets, df_aligned=pd.DataFrame(), mergeOnLeft="Rue", mergeOnRight=["L_VOIE"]):
    # make copies to avoid alteration of source dataframes
    not_aligned = df_not_aligned.copy()
    streets = df_streets.copy()
    if not df_aligned.empty:
        aligned = df_aligned.copy()
    else:
        aligned = df_aligned
        
    for rkey in mergeOnRight:
        # format to make match possible
        streets[rkey] = streets[rkey].str.lower()
        not_aligned[mergeOnLeft] = not_aligned[mergeOnLeft].str.lower()
        # merge
        merged = not_aligned.merge(streets, how="left", left_on=mergeOnLeft, right_on=rkey, suffixes=(False, False))
        # append newly aligned cases to aligned dataframe
        newly_aligned = merged[merged[rkey].notna()]
        # add column to know what alignment was on
        newly_aligned["aligned_on"] = rkey
        aligned = pd.concat([aligned, newly_aligned])
        
        # update not aligned rows
        not_aligned = merged[merged[rkey].isna()]
        not_aligned = not_aligned.drop(list(streets.columns), axis=1)
        print(f"Joining on {rkey}\n#total aligned: {len(aligned)}, newly aligned: {len(newly_aligned)}, not aligned: {len(not_aligned)}")
    
    return aligned, not_aligned

In [187]:
aligned, not_aligned = align_on_column(bottins, streets, mergeOnLeft="Rue_no_abbr", mergeOnRight=["streetname"])
aligned, not_aligned = align_on_column(df_not_aligned = not_aligned, df_streets= streets, 
                    df_aligned = aligned, mergeOnLeft="Rue_no_abbr", mergeOnRight=["streetname_short"])

C:\Users\Lea\AppData\Local\Temp/ipykernel_20448/831735043.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newly_aligned["aligned_on"] = rkey


Joining on streetname
#total aligned: 1209127, newly aligned: 1209127, not aligned: 3224261
Joining on streetname_short
#total aligned: 5553241, newly aligned: 4344114, not aligned: 1148360


In [188]:
Counter(not_aligned["Rue_no_abbr"]).most_common(100)

[('boulevard sebastopol', 13158),
 ('lafayette', 12845),
 ('boulevard magenta', 11252),
 ('grenelle saint germain', 8446),
 ('n. d. de nazareth', 7039),
 ('allemagne', 6737),
 ('boulevard montparnasse', 6084),
 ('rochechouart', 5863),
 ("avenue d'orleans", 5696),
 ('quai jemmapes', 5620),
 ('rue de flandre', 5072),
 ('j. j. rousseau', 5063),
 ('boulevard bonne nouvelle', 4951),
 ('paradis poissonniere', 4544),
 ('saint louis marais', 4218),
 ('bourbon villeneuve', 3833),
 ('boulevard de la gare', 3826),
 ('saint maur popincourt', 3823),
 ('rue lafayette', 3699),
 ('quai valmy', 3672),
 ('michodiere', 3599),
 ('grenetat', 3493),
 ('four saint germain', 3340),
 ('n. d. de lorette', 3143),
 ("rue d'angouleme", 3072),
 ('boulevard menilmontant', 3065),
 ('petit carreau', 2866),
 ('boulevard rochechouart', 2855),
 ("rue d'allemagne", 2843),
 ('vanves', 2797),
 ('albouy', 2766),
 ('avenue wagram', 2708),
 ('faubourg stmartin', 2670),
 ('grenelle saint honore', 2571),
 ('phelippeaux', 2552),


In [195]:
[x for x in streets["streetname_short"] if "magenta" in x]

['magenta', 'magenta', 'magenta']

In [190]:
streets[streets["streetname_short"]=="la fayette"]

,id,nom_voie,voie,voie_prep,nom_voie_prep,N_SQ_VO,L_VOIE,L_COURTMIN,L_LONGMIN,L_VOIE_prep,L_COURTMIN_prep,L_LONGMIN_prep,streetname,source,streetname_short
2667,NaN,NaN,NaN,NaN,NaN,750004617.0,LA FAYETTE,R. la Fayette,Rue la Fayette,la fayette,r. la fayette,rue la fayette,rue la fayette,2022,la fayette


In [191]:
aligned.head()

,Unnamed: 0,page,row,Nom,Métier,Rue,Numéro,annee,Rue_prep,Rue_no_abbr,...,L_VOIE,L_COURTMIN,L_LONGMIN,L_VOIE_prep,L_COURTMIN_prep,L_LONGMIN_prep,streetname,source,streetname_short,aligned_on
12,bpt6k6282019m,144,21,Abbatucci (Séverin),député de la Corse,place Vendôme,11-13.,1855,place vendome,place vendome,...,VENDOME,Pl. Vendôme,Place Vendôme,vendome,pl. vendome,place vendome,place vendome,2022,vendome,streetname
13,bpt6k6282019m,144,24,Abbatucci # (Th.),maître des requêtes au conseil d'Etat,pl. Vendôme,ll et 13.,1855,pl. vendome,place vendome,...,VENDOME,Pl. Vendôme,Place Vendôme,vendome,pl. vendome,place vendome,place vendome,2022,vendome,streetname
21,bpt6k6282019m,144,33,Abel-Mouillard et E. Gaspart,fab. de châles et confections pour dames,boul. St-Denis,9.,1855,boul. st denis,boulevard saint denis,...,SAINT-DENIS,Bd. Saint-Denis,Boulevard Saint-Denis,saint denis,bd. saint denis,boulevard saint denis,boulevard saint denis,2022,saint denis,streetname
25,bpt6k6282019m,144,37,Abette,tapissier,Pont-Louis-Philippe,13.,1855,pont louis philippe,pont louis philippe,...,LOUIS-PHILIPPE,Pont Louis-Philippe,Pont Louis-Philippe,louis philippe,pont louis philippe,pont louis philippe,pont louis philippe,2022,louis philippe,streetname
103,bpt6k6282019m,144,134,Adam,limonadier,avenue des Champs-Elysées,39.,1855,avenue des champs elysees,avenue des champs elysees,...,CHAMPS ELYSEES,Av. des Champs Elysées,Avenue des Champs Elysées,champs elysees,av. des champs elysees,avenue des champs elysees,avenue des champs elysees,2022,champs elysees,streetname


## Substitute words by dictionary

In [192]:
#handle "notre dame" differently (other method cannot handle spaces)

def substitute_words_by_dict(row, word_dict):
    # dict of form {"correct word": ["misspelled/abbreviated words"]}
    for correct, substitute_list in word_dict.items():
        for incorrect in substitute_list:
            if incorrect in row:
                row = row.replace(incorrect, correct)
    return row

word_dict = {"notre dame": ["n. d.", "n. d "], 
                "notre da": ["n. da"],
                "boulevard de": ["boulevard"],
                "la fayette": ["lafayette"]}

def notre_dame(row):
    if "n." in row:
        if "n. d." in row:
            return row.replace("n. d.", "notre dame")
        elif "n. da" in row:
            return row.replace("n.", "notre")
        elif "n. d " in row:
            return row.replace("n. d", "notre dame")
        else:
            return row
    else:
        return row

print("#abbr. in data before notre dame subst.:", len(set([get_abbreviation(street) for street in list(bottins["Rue_no_abbr"])])))
# substitute abbreviations
not_aligned["Rue_no_abbr"] = not_aligned["Rue_no_abbr"].apply(substitute_words_by_dict, args=(word_dict,))
# see how many abbreviations are left
print("#abbr. in data after notre dame subst.:", len(set([get_abbreviation(street) for street in list(bottins["Rue_no_abbr"])])))

#abbr. in data before notre dame subst.: 37883
#abbr. in data after notre dame subst.: 37883


In [193]:
aligned, not_aligned = align_on_column(df_not_aligned = not_aligned, df_streets= streets, 
                    df_aligned = aligned, mergeOnLeft="Rue_no_abbr", mergeOnRight=["streetname"])

C:\Users\Lea\AppData\Local\Temp/ipykernel_20448/831735043.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newly_aligned["aligned_on"] = rkey


Joining on streetname
#total aligned: 5603484, newly aligned: 50243, not aligned: 1098117


In [194]:
Counter(not_aligned["Rue_no_abbr"]).most_common(100)

[('la fayette', 12845),
 ('grenelle saint germain', 8446),
 ('notre dame de nazareth', 7122),
 ('allemagne', 6737),
 ('boulevard de montparnasse', 6084),
 ('rochechouart', 5863),
 ("avenue d'orleans", 5696),
 ('quai jemmapes', 5620),
 ('rue de flandre', 5072),
 ('j. j. rousseau', 5063),
 ('paradis poissonniere', 4544),
 ('saint louis marais', 4218),
 ('bourbon villeneuve', 3833),
 ('boulevard de de la gare', 3826),
 ('saint maur popincourt', 3823),
 ('quai valmy', 3672),
 ('michodiere', 3599),
 ('grenetat', 3493),
 ('four saint germain', 3340),
 ('notre dame de lorette', 3228),
 ("rue d'angouleme", 3072),
 ('petit carreau', 2866),
 ('boulevard de rochechouart', 2855),
 ("rue d'allemagne", 2843),
 ('vanves', 2797),
 ('albouy', 2766),
 ('avenue wagram', 2708),
 ('faubourg stmartin', 2670),
 ('grenelle saint honore', 2571),
 ('phelippeaux', 2552),
 ('notre dame des victoires', 2516),
 ('la harpe', 2513),
 ('fontaine saint georges', 2485),
 ('cite trevise', 2459),
 ('chaussee d’antin', 236

In [197]:
not_aligned

,Unnamed: 0,page,row,Nom,Métier,Rue,Numéro,annee,Rue_prep,Rue_no_abbr
0,bpt6k6282019m,144,0,Aaron,bronzes,passage Choiseal,72 et 74.,1855,passage choiseal,passage choiseal
1,bpt6k6282019m,144,13,Abault et Coudray,charpentiers,Corbeau,23.,1855,corbeau,corbeau
2,bpt6k6282019m,144,14,Abault (Paul),libraire,quai des Angustins,9.,1855,quai des angustins,quai des angustins
3,bpt6k6282019m,144,15,Abavid,vins,Beaujolais-da-Temple,7.,1855,beaujolais da temple,beaujolais da temple
4,bpt6k6282019m,144,16,Abazaer (Are),cristaux et porcelaines,Pei.Ecuries,26.,1855,pei.ecuries,pei.ecuries
...,...,...,...,...,...,...,...,...,...,...
1148355,bpt6k9780089g,1607,206,Zurbrugg,ébéniste,pass. de la Main-d'Or,4.,1922,pass. de la main d'or,passerelle de la main d'or
1148356,bpt6k9780089g,1607,212,Zurconi (P.),fourreur,quai de l'Hôtel-deVille,46.,1922,quai de l'hotel deville,quai de l'hotel deville
1148357,bpt6k9780089g,1607,214,Zuretti,vins et hôtel,r. des Graviliers,80.,1922,r. des graviliers,rue des graviliers
1148358,bpt6k9780089g,1607,220,Zurfluh (A.),éditeur de musique,r. des StsPères,14.,1922,r. des stsperes,rue des stsperes
